### Notebook Setup

In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from IPython.display import display, HTML

In [2]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
%%R

# My commonly used R imports

require('ggplot2')
require('readr')
require('extrafont')
require('dplyr')
require('reshape2')
# require('theme538')

R[write to console]: Loading required package: ggplot2



R[write to console]: Loading required package: readr

R[write to console]: Loading required package: extrafont

R[write to console]: Registering fonts with R

R[write to console]: Loading required package: dplyr

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: Loading required package: reshape2



In [4]:
# set maximum number of columns to display
pd.set_option('display.max_columns', None)

# Refrigerator Analysis

In [5]:
epa_ref = pd.read_csv('epa_residential_refrigerators.csv')
epa_freeze =  pd.read_csv('epa_residential_freezers.csv')
doe_ref = pd.read_csv('doe-refrigerator.csv')

In [6]:
epa_ref_freeze = pd.concat([epa_ref, epa_freeze])
# epa_ref_freeze.to_csv('epa_refrigerator_freezer_combined.csv', index=False)

In [7]:
# Analyze energy efficiency of refrigerators and freezers by brand.

epa_ref_freeze.head()

,ENERGY STAR Unique ID,Brand Name,Model Number,Additional Model Information,UPC,Type,Product Class,Defrost Type,Compact,Built-in,Thru the Door Dispenser,Ice Maker,Counter Depth,Height (in),Width (in),Capacity (Total Volume) (ft3),Adjusted Volume (ft3),Annual Energy Use (kWh/yr),US Federal Standard (kWh/yr),Percent Less Energy Use than US Federal Standard,Refrigerant Type,Refrigerant with GWP,Connected Functionality,Connects Using,Connected Functionality Tested,Communication Standard Application Layer,Direct on-premises Open-standard Based Interconnection,Date Available On Market,Date Certified,Markets,CB Model Identifier,Meets ENERGY STAR Most Efficient 2023 Criteria
0,2369304,Arctic Wind,2ATMW212E,NaN,NaN,Top Freezer,3 - Refrigerator-freezers - automatic defrost ...,Automatic,No,No,No,No,No,67.0,33.0,21.0,25.6,396,440.0,10.0,NaN,NaN,No,NaN,NaN,NaN,NaN,10/30/2020,11/09/2020,"United States, Canada",ES_1121980_2ATMW212E_11102020012636_1596636,Yes
1,2397747,Arctic Wind,2AW1BF26A,",2AW1SLF26A,",850012484240;850012484370,Compact Refrigerator,11A - Compact All-Refrigerator - manual defrost,Manual,Yes,No,No,No,No,27.4,17.5,2.6,2.6,215,239.0,10.0,NaN,NaN,No,NaN,NaN,NaN,NaN,08/15/2022,06/23/2022,"United States, Canada",ES_1121980_2AW1BF26A_06202022134236_2692056,No
2,2393669,Arctic Wind,2AW1SLF44A,NaN,850012484400,Compact Refrigerator,11A - Compact All-Refrigerator - manual defrost,Manual,Yes,No,No,No,No,33.1,18.7,4.4,4.4,228,254.0,10.0,NaN,NaN,No,NaN,NaN,NaN,NaN,04/15/2022,03/08/2022,"United States, Canada",ES_1121980_2AW1SLF44A_03012022142251_5311380,No
3,2370277,Arctic Wind,ABM*171E,",,The symbol *, maybe replaced by A-Z, means d...",NaN,Bottom Freezer,5 - Refrigerator-freezers - automatic defrost ...,Automatic,No,No,No,No,No,69.1,31.3,17.1,20.9,451,502.0,10.0,R-600a,R-600a (GWP:3 | Lower GWP),No,NaN,NaN,NaN,NaN,11/23/2020,11/25/2020,"United States, Canada",ES_1121980_ABM*171E_112520200126738_9572853,No
4,2370276,Arctic Wind,ABM*171E*,",,The symbol *, maybe replaced by A-Z, means d...",NaN,Bottom Freezer,5 - Refrigerator-freezers - automatic defrost ...,Automatic,No,No,No,No,No,69.1,31.3,17.1,20.9,451,502.0,10.0,R-600a,R-600a (GWP:3 | Lower GWP),No,NaN,NaN,NaN,NaN,11/23/2020,11/25/2020,"United States, Canada",ES_1121980_ABM*171E*_11252020012627_7795251,No


In [8]:
epa_ref_freeze['energy_per_volume'] = epa_ref_freeze['Annual Energy Use (kWh/yr)'] / epa_ref_freeze['Adjusted Volume (ft3)']

In [12]:
# calculate the average energy efficiency of each brand
least_efficient = epa_ref_freeze.groupby('Brand Name')['energy_per_volume'].mean().sort_values(ascending=False).reset_index().head(10)
most_efficienct = epa_ref_freeze.groupby('Brand Name')['energy_per_volume'].mean().sort_values(ascending=True).reset_index().head(10)

least_efficient.to_csv('least_efficient.csv', index=False)
most_efficienct.to_csv('most_efficienct.csv', index=False)

In [13]:
least_efficient

,Brand Name,energy_per_volume
0,ECOFLOW,186.923077
1,Mini Bar,156.871795
2,White-Westinghouse,129.375000
3,RVISION,111.149733
4,Danby or Danby Designer,103.929924
5,Willz,103.225806
6,DOMETIC,102.889321
7,IGLOO,100.355240
8,Antarctic Star,99.218750
9,Lorell,98.402778


In [14]:
most_efficienct

,Brand Name,energy_per_volume
0,Asko,9.599462
1,Gallery,14.264577
2,Elisii,15.330535
3,CTM,16.086957
4,ALPHA,16.147541
5,SANKEY,16.272111
6,Gladiator,16.460674
7,Danby or Danby Designer or Danby Premiere or S...,16.466667
8,Danby Premiere,16.466667
9,Conservator,16.705643


In [7]:
# Find all models from the EPA are in the DOE, using REGEX

def regexify_model_number(model_number):
    model_number_regex = re.sub(r'\*', '.', model_number)
    model_number_regex = re.sub(r'\#', '.', model_number_regex)
    return model_number_regex

epa_ref_freeze['model_number_modified'] = epa_ref_freeze['Model Number'].apply(regexify_model_number)
doe_ref['individual_model_number_modified'] = doe_ref['Individual_Model_Number_Covered_by_Basic_Model_m'].apply(regexify_model_number)

In [8]:
def find_epa_model_number_in_doe_database(model_number):
    return doe_ref['individual_model_number_modified'].str.contains(model_number, regex=True).any()

In [12]:
epa_models_in_doe_database = epa_ref_freeze[epa_ref_freeze.model_number_modified.apply(find_epa_model_number_in_doe_database)]
epa_models_in_doe_database.head()
# epa_models_in_doe_database.to_csv('epa_ref_freezer_models_in_doe_database.csv', index=False)

,ENERGY STAR Unique ID,Brand Name,Model Number,Additional Model Information,UPC,Type,Product Class,Defrost Type,Compact,Built-in,Thru the Door Dispenser,Ice Maker,Counter Depth,Height (in),Width (in),Capacity (Total Volume) (ft3),Adjusted Volume (ft3),Annual Energy Use (kWh/yr),US Federal Standard (kWh/yr),Percent Less Energy Use than US Federal Standard,Refrigerant Type,Refrigerant with GWP,Connected Functionality,Connects Using,Connected Functionality Tested,Communication Standard Application Layer,Direct on-premises Open-standard Based Interconnection,Date Available On Market,Date Certified,Markets,CB Model Identifier,Meets ENERGY STAR Most Efficient 2023 Criteria,model_number_modified
19,2354266,Avanti,AR17T0W,",AR17T1B,; ,AR17T3S,",NaN,Compact Refrigerator,13A - Compact All-Refrigerators - automatic de...,Automatic,Yes,No,No,No,No,20.2,17.7,1.7,1.7,233,275.0,15.0,NaN,NaN,No,NaN,NaN,NaN,NaN,01/01/2020,01/30/2020,United States,ES_92257_AR17T1B_01102020102044_8042261,No,AR17T0W
20,2309224,Avanti,AR17T1B,",AR17T0W,; ,AR17T3S,",NaN,Compact Refrigerator,13A - Compact All-Refrigerators - automatic de...,Automatic,Yes,No,No,No,No,20.1,17.7,1.7,1.7,238,275.0,13.0,NaN,NaN,No,NaN,NaN,NaN,NaN,11/15/2017,01/16/2018,"United States, Canada",ES_92257_AR17T1B_11232017092331_4355584,No,AR17T1B
21,2216363,Avanti,AR2406W,",AR2416B,",NaN,Compact Refrigerator,13A - Compact All-Refrigerators - automatic de...,Automatic,Yes,No,No,No,No,25.0,18.5,2.2,2.2,230,279.0,18.0,NaN,NaN,No,NaN,NaN,NaN,NaN,07/21/2014,07/22/2014,United States,ES_92257_AR2406W_07212014164849_4965174,No,AR2406W
22,2303594,Avanti,AR24T3S,NaN,NaN,Compact Refrigerator,13A - Compact All-Refrigerators - automatic de...,Automatic,Yes,No,No,No,No,24.9,18.5,2.4,2.4,238,281.0,15.0,NaN,NaN,No,NaN,NaN,NaN,NaN,09/01/2017,09/14/2017,United States,ES_92257_AR24T3S_09122017132724_1491662,No,AR24T3S
23,2353672,Avanti,AR24T3S,NaN,NaN,Compact Refrigerator,13A - Compact All-Refrigerators - automatic de...,Automatic,Yes,No,No,No,No,24.8,17.7,2.4,2.4,238,281.0,15.0,NaN,NaN,No,NaN,NaN,NaN,NaN,01/01/2020,01/16/2020,United States,ES_92257_AR24T3S_01102020153313_9034234,No,AR24T3S


In [10]:
epa_models_not_in_doe_database = epa_ref_freeze[~epa_ref_freeze.model_number_modified.apply(find_epa_model_number_in_doe_database)]
# epa_models_not_in_doe_database.to_csv('epa_ref_freezer_models_not_in_doe_database.csv', index=False)

In [11]:
def find_doe_models_in_energy_star(model_number):
    return epa_ref_freeze['model_number_modified'].str.contains(model_number, regex=True).any()

# DOE models that appear in EPA ENERGY STAR dataset
doe_ref_energy_star = doe_ref[doe_ref.individual_model_number_modified.apply(find_doe_models_in_energy_star)]

# DOE models that do not appear in EPA ENERGY STAR dataset
doe_ref_non_energy_star = doe_ref[~doe_ref.individual_model_number_modified.apply(find_doe_models_in_energy_star)]

In [12]:
# doe_ref_energy_star.to_csv('doe_models_in_energy_star_database.csv', index=False)
# doe_ref_non_energy_star.to_csv('doe_models_not_in_energy_star_database.csv', index=False)

# TV Analysis

In [13]:
tv_epa = pd.read_csv('epa-tv.csv')
tv_doe = pd.read_csv('doe-tv.csv')

In [14]:
tv_epa.head()

,ENERGY STAR Unique ID,ENERGY STAR Partner,Brand Name,Model Name,Model Number,Additional Model Information,UPC,Product Type,Application,Display Type,Backlight Technology Type,Diagonal Viewable Screen Size (in.),Screen Area (sq. in.),Native Horizontal Resolution (pixels),Native Vertical Resolution (pixels),Resolution Format,High Contrast Ratio (HCR) Display,Physical Data Ports Available,Ethernet Supported,Low Power Wireless Technologies Supported,Features,Automatic Brightness Control,Is Automatic Brightness Control Enabled by Default in the Default SDR Preset Picture Setting When Television is Shipped?,Average On Mode Power Consumption for Certification (watts),Maximum Average On Mode Power for Certification (watts),Reported On Mode Power (per the Federal Test Procedure) (watts),Reported Annual Energy Consumption (kWh),Power Consumption in Standby Mode when Not Connected to a Network (watts),Power Consumption in Standby Mode when Connected to a Network (watts),"Reported Standby-Active, Low Mode Power (watts)",Date Available On Market,Date Certified,Markets,CB Model Identifier
0,2405713,"Top Victory Electronics (Taiwan) Co., Ltd.",PHILIPS,50HFL4518U/27,50HFL4518U/27,NaN,871886300399,Television (TV),Consumer,LCD,Direct-lit LED,49.5,1047.00,3840,2160,4K (UHD),No,"Universal Serial Bus (USB),HDMI","Gigabit Ethernet (1000 Mbit/s),Gigabit Energy ...",None,"High Dynamic Range Upscaling,Full Network Conn...",No,NaN,66.90,71.41,66.9,114.15,0.35,0.47,0.47,11/15/2021,11/21/2022,United States,ES_1065104_50HFL4518U/27_11212022142452I59_734...
1,2405721,"Top Victory Electronics (Taiwan) Co., Ltd.",PHILIPS,55HFL4518U/27,55HFL4518U/27,NaN,871886300405,Television (TV),Consumer,LCD,Direct-lit LED,54.6,1275.70,3840,2160,4K (UHD),No,"Universal Serial Bus (USB),HDMI","Gigabit Ethernet (1000 Mbit/s),Gigabit Energy ...",None,"High Dynamic Range Upscaling,Full Network Conn...",No,NaN,93.20,94.11,93.2,150.27,0.40,0.51,0.51,11/18/2022,11/23/2022,United States,ES_1065104_55HFL4518U/27_1123202292455I14_7349927
2,2401931,"Top Victory Electronics (Taiwan) Co., Ltd.",PHILIPS,55HFL5214U/27,55HFL5214U/27,"55BFL2214/27,55BFL2214/27,",871886300214;871886300276,Television (TV),Consumer,LCD,Direct-lit LED,54.5,1275.70,3840,2160,4K (UHD),No,"Universal Serial Bus (USB),HDMI",Gigabit Ethernet (1000 Mbit/s),None,None,No,NaN,83.23,93.08,90.8,129.76,0.40,0.79,0.79,12/20/2021,09/19/2022,United States,ES_1065104_55HFL5214U/27_09192022122420I33_734...
3,2404180,"Top Victory Electronics (Taiwan) Co., Ltd.",PHILIPS,65HFL5214U/27,65HFL5214U/27,",65BFL2214/27,65BFL2214/27",871886300221;871886300283,Television (TV),Consumer,LCD,Direct-lit LED,64.5,1779.10,3840,2160,4K (UHD),No,"Universal Serial Bus (USB),HDMI",Gigabit Ethernet (1000 Mbit/s),None,None,No,NaN,126.80,129.13,126.8,159.42,0.43,0.83,0.83,12/21/2021,10/20/2022,United States,ES_1065104_65HFL5214U/27_1020202214246I20_7349927
4,2407358,Xitrix Computer Corporation,XITRIX,XPN-DSA3250,XPN-DSA3250,NaN,NaN,Television (TV),Consumer,LCD,Direct-lit LED,32.0,424.18,768,1366,Other,No,"Universal Serial Bus (USB),Other,HDMI",None,None,None,No,NaN,34.40,42.80,34.4,64.37,0.22,0.22,0.22,01/05/2023,01/26/2023,"United States, Canada",ES_1058575_XPN-DSA3250_01162023112847_6901971


In [15]:
# Show number of rows in tv_epa and tv_doe
len(tv_epa)


45

In [16]:
tv_doe.head()

,Brand_Name_s__s,Individual_Model_Number_m,Screen_Size__diagonal_in_inches__d,Power_Consumed_in_On_Mode__watts__d,Power_Consumed_in_Standby_Passive_Mode__watts__d,Power_Consumed_in_Standby_Active__Low_Mode__watts___if_applicable__d,Power_Consumed_in_Off_Mode__watts___if_applicable__d,Annual_Energy_Consumption__kWh_per_year__d,Link_to_FTC_EnergyGuide_Label_s,Is_the_Certification_for_this_Basic_Model_Based_on_a_Waiver_of_DOE_s_Test_Procedure_Requirements__s
0,Hisense,75U78H,75.0,178.0,0.3,0.4,NaN,327.0,https://www.hisense-usa.com/support/models,No
1,Hisense,75U7H+(+=0-9 or A-Z or Blank),75.0,178.0,0.3,0.4,NaN,327.0,https://www.hisense-usa.com/support/models,No
2,Hisense,75U7+H+(+=0-9 or A-Z or Blank),75.0,178.0,0.3,0.4,NaN,327.0,https://www.hisense-usa.com/support/models,No
3,Hisense,85A6+H++(+=0-9 or A-Z or Blank),85.0,241.0,0.3,0.3,NaN,441.0,https://www.hisense-usa.com/support/models,No
4,Hisense,85A7G+(+=0-9 or A-Z or Blank),85.0,241.0,0.3,0.3,NaN,441.0,https://www.hisense-usa.com/support/models,No


In [17]:
len(tv_doe)

4159

In [18]:
# Replace all '+' with '.' in the model number
tv_epa['model_number_modified'] = tv_epa['Model Number'].str.replace('+', '.')

In [19]:
def regexify_model_number(model_number):
    model_number_regex = re.sub(r'\*', '.', model_number)
    model_number_regex = re.sub(r'\#', '.', model_number_regex)
    model_number_regex = model_number_regex.replace('+', '.')
    return model_number_regex

In [20]:
tv_epa['model_number_modified'] = tv_epa['Model Number'].apply(regexify_model_number)
tv_doe['individual_model_number_modified'] = tv_doe['Individual_Model_Number_m'].apply(regexify_model_number)

In [21]:
# Remove all string between parentheses
tv_doe['individual_model_number_modified'] = tv_doe['individual_model_number_modified'].str.replace(r"\(.*\)","")

In [22]:
def find_epa_model_number_in_doe_database(model_number):
    return tv_doe['individual_model_number_modified'].str.contains(model_number, regex=True).any()

tv_epa_models_in_doe_database = tv_epa[tv_epa.model_number_modified.apply(find_epa_model_number_in_doe_database)]
len(tv_epa_models_in_doe_database)
tv_epa_models_in_doe_database.to_csv('epa_tv_models_in_doe_database.csv', index=False)

In [23]:
tv_epa_models_not_in_doe_database = tv_epa[~tv_epa.model_number_modified.apply(find_epa_model_number_in_doe_database)]
tv_epa_models_not_in_doe_database.to_csv('epa_tv_models_not_in_doe_database.csv', index=False)
len(tv_epa_models_not_in_doe_database)

10

In [24]:
def find_doe_models_in_energy_star(model_number):
    return tv_epa['model_number_modified'].str.contains(model_number, regex=True).any()

# DOE models that appear in EPA ENERGY STAR dataset
tv_doe_energy_star = tv_doe[tv_doe.individual_model_number_modified.apply(find_doe_models_in_energy_star)]
# tv_doe_energy_star.to_csv('doe_tv_models_in_energy_star_database.csv', index=False)

# DOE models that do not appear in EPA ENERGY STAR dataset
tv_doe_non_energy_star = tv_doe[~tv_doe.individual_model_number_modified.apply(find_doe_models_in_energy_star)]
tv_doe_non_energy_star.to_csv('doe_tv_models_not_in_energy_star_database.csv', index=False)

In [25]:
print(len(tv_doe_energy_star))
print(len(tv_doe_non_energy_star))

46
4113


# Residential Clothes washer Analysis

In [26]:
dishwasher_epa = pd.read_csv("epa_dishwasher.csv")
dishwasher_doe = pd.read_csv("doe_dishwasher.csv")

In [27]:
dishwasher_epa.head()

,ENERGY STAR Unique ID,Brand Name,Model Number,Additional Model Information,UPC,Type,Width (inches),Depth (inches),Capacity - Maximum Number of Place Settings,Soil-Sensing Capability,Tub Material,Drying Method,Additional Product Features,Annual Energy Use (kWh/yr),US Federal Standard (kWh/yr),% Better than US Federal Standard (kWh/yr),Water Use (gallons/cycle),US Federal Standard (gallons/cycle),% Better than US Federal Standard (gallons/cycle),Connected Capable,Connects Using,Communication Hardware Architecture,DR Protocol,Direct on-premises Open-standard Based Interconnection,Date Available On Market,Date Certified,Markets,CB Model Identifier,Meets ENERGY STAR Most Efficient 2023 Criteria
0,2386941,Bosch,SHEM63W5**,",SHEM63W5**,First ""*"" is for the color, 2nd ""*...",825225922223;825225922438;825225922445,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,269,307.0,12.0,2.9,5.0,42.0,No,NaN,NaN,NaN,NaN,11/09/2016,10/26/2021,"United States, Canada",ES_31649_SHEM63W5**_10272021101706_1474573,No
1,2282867,Bosch,SHEM78W5#*,",SHEM78W5#*,""#"" is for the color, ""*"" is for ...",825225923046;825225923053;825225923060,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,269,307.0,12.0,2.9,5.0,42.0,No,NaN,NaN,NaN,NaN,11/09/2016,10/26/2016,"United States, Canada",ES_31649_SHEM78W5#*_10272016211527_9332087,No
2,2284086,Bosch,SHEM78WH#*,NaN,825225923954,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239,307.0,22.0,2.9,5.0,42.0,No,NaN,NaN,NaN,NaN,10/27/2016,11/15/2016,"United States, Canada",ES_31649_SHEM78WH#*_10272016142033_5994378,No
3,2338251,Bosch,SHEM78Z5#*,",SHEM78Z5#*,1st # - Color Code, 2nd * - Countr...",825225958567;825225958574;825225958581,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,269,307.0,12.0,3.5,5.0,30.0,No,NaN,NaN,NaN,NaN,05/30/2019,05/15/2019,"United States, Canada",ES_31649_SHEM78Z5#*_05142019152636_2580377,No
4,2338252,Bosch,SHEM78ZH#*,",SHEM78ZH#*,1st # - Color Code, 2nd * - Countr...",825225959434,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,269,307.0,12.0,3.5,5.0,30.0,No,NaN,NaN,NaN,NaN,05/30/2019,05/15/2019,"United States, Canada",ES_31649_SHEM78ZH#*_05142019152636_8975946,No


In [28]:
len(dishwasher_epa)

1060

In [29]:
dishwasher_doe.head()

,Brand_Name_s__s,Product_Group_Code_Description_s,Basic_Model_Number_m,Individual_Model_Number_Covered_by_Basic_Model_m,Annual_Energy_Use__Kilowatt_Hours_Year__d,Water_Consumption__Gallons_Cycle__d,Capacity__Number_of_Place_Settings__d,Is_the_Certification_for_this_Basic_Model_Based_on_a_Waiver_of_DOE_s_Test_Procedure_Requirements__s,Is_the_Certification_based_upon_any_Exception_Relief_from_an_Applicable_Standard_by_DOE_s_Office_of_Hearing_and_Appeals__s,Link_to_FTC_EnergyGuide_Label_s
0,LG,Standard size dishwasher,LDP681***,LDP681***,238.0,2.9,15.0,No,No,https://www.lg.com/us/dishwashers
1,LG,Standard size dishwasher,LDP679#**,LDP679#**,238.0,2.9,15.0,No,No,https://www.lg.com/us/dishwashers
2,hOme,Compact dishwasher,WQP6-3601-US,WQP6-3602G-US,203.0,3.1,6.0,No,No,By annual report date
3,hOme,Compact dishwasher,WQP6-3601-US,HME010393N,203.0,3.1,6.0,No,No,By annual report date
4,Farberware,Compact dishwasher,WQP6-3601-US,FCD06ASWWHC,203.0,3.1,6.0,No,No,By annual report date


In [30]:
len(dishwasher_doe)

1269

In [31]:
dishwasher_epa['model_number_modified'] = dishwasher_epa['Model Number'].apply(regexify_model_number)
dishwasher_doe['individual_model_number_modified'] = dishwasher_doe['Individual_Model_Number_Covered_by_Basic_Model_m'].apply(regexify_model_number)

In [32]:
# find epa model number in doe database
def find_epa_model_number_in_doe_database(model_number):
    return dishwasher_doe['individual_model_number_modified'].str.contains(model_number, regex=True).any()

dishwasher_epa_models_in_doe_database = dishwasher_epa[dishwasher_epa.model_number_modified.apply(find_epa_model_number_in_doe_database)]
dishwasher_epa_models_in_doe_database.to_csv('epa_dishwasher_models_in_doe_database.csv', index=False)

dishwasher_epa_models_not_in_doe_database = dishwasher_epa[~dishwasher_epa.model_number_modified.apply(find_epa_model_number_in_doe_database)]
dishwasher_epa_models_not_in_doe_database.to_csv('epa_dishwasher_models_not_in_doe_database.csv', index=False)



In [33]:
print(len(dishwasher_epa_models_in_doe_database))
print(len(dishwasher_epa_models_not_in_doe_database))

696
364


In [34]:
def find_doe_models_in_energy_star(model_number):
    return dishwasher_epa['model_number_modified'].str.contains(model_number, regex=True).any()

# DOE models that appear in EPA ENERGY STAR dataset
dishwasher_doe_energy_star = dishwasher_doe[dishwasher_doe.individual_model_number_modified.apply(find_doe_models_in_energy_star)]
dishwasher_doe_energy_star.to_csv('doe_dishwasher_models_in_energy_star_database.csv', index=False)

# DOE models that do not appear in EPA ENERGY STAR dataset
dishwasher_doe_non_energy_star = dishwasher_doe[~dishwasher_doe.individual_model_number_modified.apply(find_doe_models_in_energy_star)]
dishwasher_doe_non_energy_star.to_csv('doe_dishwasher_models_not_in_energy_star_database.csv', index=False)

print(len(dishwasher_doe_energy_star))
print(len(dishwasher_doe_non_energy_star))

1016
253


# Clothes Washer Analysis

In [35]:
clotheswasher_epa = pd.read_csv("epa_clotheswasher.csv")
clotheswasher_doe = pd.read_csv("doe_clotheswasher.csv")

print(len(clotheswasher_epa))
print(len(clotheswasher_doe))

344
1263


In [36]:
clotheswasher_epa.head()

,ENERGY STAR Unique ID,Brand Name,Model Number,Additional Model Information,UPC,Load Configuration,Special Type,Additional Washer Features,Intended Market,Volume (cu. ft.),Height (inches),Width (inches),Depth (inches),Integrated Modified Energy Factor (IMEF),US Federal Standard (IMEF),Annual Energy Use (kWh/yr),Integrated Water Factor (IWF),US Federal Standard (IWF),Annual Water Use (gallons/yr),Drum Capacity for the dryer in a Combination All-in-One Washer-Dryer,Heat Pump Technology for the dryer in a Combination All-in-One Washer-Dryer,Combined Energy Factor (CEF) for the dryer in a Combination All-in-One Washer-Dryer,Estimated Annual Energy Use (kWh/yr) for the dryer in a Combination All-in-One Washer-Dryer,Estimated Energy Test Cycle Time (min) for the dryer in a Combination All-in-One Washer-Dryer,Energy Test Cycle Information for the dryer in a Combination All-in-One Washer-Dryer,Calculated Combined Energy Factor - Max Dryness Setting (lbs/kWh) for the dryer in a Combination All-in-One Washer-Dryer,Additional Dryer Features for the dryer in a Combination All-in-One Washer-Dryer,Connected,Connects Using,Communication Standard Application Layer,Direct on-premises Open-standard Based Interconnection,Paired ENERGY STAR Clothes Dryer Available,Paired ENERGY STAR Clothes Dryer ENERGY STAR Model Identifier,Date Available On Market,Date Certified,Markets,ENERGY STAR Model Identifier,Meets ENERGY STAR Most Efficient 2023 Criteria
0,2374994,GE,GTW490BMR***,NaN,757638370103,Top Load,NaN,Gentle Cycle,Residential,4.1,NaN,26.8,NaN,2.06,1.29,156,4.3,8.4,5252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,No,NaN,02/22/2021,02/22/2021,Canada,ES_92277_GTW490BMR***_031820210717386_1699983,No
1,2331678,GE,GTW500A*N***,",GTW500ASNWS,Color: White / Silver",084691845638,Top Load,NaN,"Gentle Cycle,Delayed Start",Residential,4.6,NaN,27.0,NaN,2.06,1.29,175,4.3,8.4,5860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,12/17/2018,12/05/2018,"United States, Canada",ES_1123206_GTW500A*N***_12052018194545_9145154,No
2,2370952,GE,GTW500A*N***,",GTW500ASNWS,",084691845638,Top Load,NaN,Gentle Cycle,Residential,4.6,NaN,27.0,NaN,2.06,1.29,175,4.3,8.4,5860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,04/19/2021,12/16/2020,"United States, Canada",ES_1123206_GTW500A*N***_121520200855585_6243535,No
3,2370953,GE,GTW540A*P***,",GTW540ASPWS,",084691870944,Top Load,NaN,"Sanitize Option,Gentle Cycle",Residential,4.6,NaN,27.0,NaN,2.06,1.29,175,4.3,8.4,5860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,12/22/2020,12/16/2020,"United States, Canada",ES_1123206_GTW540A*P***_12152020085595_7471356,No
4,2375000,GE,GTW550BMR***,NaN,757638370127,Top Load,NaN,Gentle Cycle,Residential,4.3,NaN,26.8,NaN,2.08,1.29,164,3.7,8.4,4683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,No,NaN,02/22/2021,02/22/2021,Canada,ES_92277_GTW550BMR***_031820210717165_7444530,No


In [37]:
clotheswasher_doe.head()

,Brand_Name_s__s,Product_Group_Code_Description_s,Basic_Model_Number_m,Individual_Model_Number_Covered_by_Basic_Model_m,Capacity__Cubic_Feet__d,Integrated_Modified_Energy_Factor__cubic_feet_per_kilowatt_hour_per_cycle__d,Integrated_Water_Factor__gallons_per_cycle_per_cubic_foot__d,Corrected_Remaining_Moisture_Content_____d,Is_the_Certification_for_this_Basic_Model_Based_on_a_Waiver_of_DOE_s_Test_Procedure_Requirements__s,Is_the_Certification_based_upon_any_Exception_Relief_from_an_Applicable_Standard_by_DOE_s_Office_of_Hearing_and_Appeals__s,Link_to_FTC_EnergyGuide_Label_s
0,Samsung,"Top-loading, standard (1.6 ft3 or greater capa...",WA45T34**A*,WA45T34**A*,4.5,1.57,6.5,39.9,No,No,https://images.samsung.com/is/content/samsung/...
1,Samsung,"Top-loading, standard (1.6 ft3 or greater capa...",WA44A3405A*,WA44A3405A*,4.4,1.57,6.5,39.7,No,No,https://images.samsung.com/is/content/samsung/...
2,Samsung,"Front-loading, standard (1.6 ft3 or greater ca...",WF50A88**A*,WF50A88**A*,5.0,3.10,2.9,39.2,No,No,https://images.samsung.com/is/content/samsung/...
3,Samsung,"Front-loading, standard (1.6 ft3 or greater ca...",WF50A86**A*,WF50A86**A*,5.0,3.10,2.9,29.7,No,No,https://images.samsung.com/is/content/samsung/...
4,Samsung,"Front-loading, standard (1.6 ft3 or greater ca...",WV60A99**A*-L,WV60A99**A*-L,5.0,2.92,2.9,29.7,No,No,https://images.samsung.com/is/content/samsung/...


In [38]:
clotheswasher_doe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1263 entries, 0 to 1262
Data columns (total 11 columns):
 #   Column                                                                                                                      Non-Null Count  Dtype  
---  ------                                                                                                                      --------------  -----  
 0   Brand_Name_s__s                                                                                                             1263 non-null   object 
 1   Product_Group_Code_Description_s                                                                                            1263 non-null   object 
 2   Basic_Model_Number_m                                                                                                        1263 non-null   object 
 3   Individual_Model_Number_Covered_by_Basic_Model_m                                                                           

In [39]:
clotheswasher_doe['Individual_Model_Number_Covered_by_Basic_Model_m']

0         WA45T34**A*
1         WA44A3405A*
2         WF50A88**A*
3         WF50A86**A*
4       WV60A99**A*-L
            ...      
1258     WFTV10733XC*
1259       WM98220SX*
1260        LW2427***
1261           TNQ41W
1262       WFW6605M**
Name: Individual_Model_Number_Covered_by_Basic_Model_m, Length: 1263, dtype: object

In [40]:
def regexify_model_number(model_number):
    model_number_regex = re.sub(r'\*', '.', str(model_number))
    model_number_regex = re.sub(r'\#', '.', str(model_number_regex))
    return model_number_regex
# 
clotheswasher_epa['model_number_modified'] = clotheswasher_epa['Model Number'].apply(regexify_model_number)
clotheswasher_doe['individual_model_number_modified'] = clotheswasher_doe['Individual_Model_Number_Covered_by_Basic_Model_m'].apply(regexify_model_number)

In [41]:
# find epa model number in doe database
def find_epa_model_number_in_doe_database(model_number):
    return clotheswasher_doe['individual_model_number_modified'].str.contains(model_number, regex=True).any()

clotheswasher_epa_models_in_doe_database = clotheswasher_epa[clotheswasher_epa.model_number_modified.apply(find_epa_model_number_in_doe_database)]
clotheswasher_epa_models_in_doe_database.to_csv('epa_clotheswasher_models_in_doe_database.csv', index=False)

clotheswasher_epa_models_not_in_doe_database = clotheswasher_epa[~clotheswasher_epa.model_number_modified.apply(find_epa_model_number_in_doe_database)]
clotheswasher_epa_models_not_in_doe_database.to_csv('epa_clotheswasher_models_not_in_doe_database.csv', index=False)

In [42]:
print(len(clotheswasher_epa_models_in_doe_database))
print(len(clotheswasher_epa_models_not_in_doe_database))

271
73


In [43]:
# Find doe model number in epa database
def find_doe_models_in_energy_star(model_number):
    return clotheswasher_epa['model_number_modified'].str.contains(model_number, regex=True).any()

# DOE models that appear in EPA ENERGY STAR dataset
clotheswasher_doe_energy_star = clotheswasher_doe[clotheswasher_doe.individual_model_number_modified.apply(find_doe_models_in_energy_star)]
clotheswasher_doe_energy_star.to_csv('doe_clotheswasher_models_in_energy_star_database.csv', index=False)

# DOE models that do not appear in EPA ENERGY STAR dataset
clotheswasher_doe_non_energy_star = clotheswasher_doe[~clotheswasher_doe.individual_model_number_modified.apply(find_doe_models_in_energy_star)]
clotheswasher_doe_non_energy_star.to_csv('doe_clotheswasher_models_not_in_energy_star_database.csv', index=False)

In [44]:
print(len(clotheswasher_doe_energy_star))
print(len(clotheswasher_doe_non_energy_star))

452
811


# Room AC Analysis

In [46]:
room_ac_epa = pd.read_csv('epa_room_ac.csv')
room_ac_doe = pd.read_csv('doe_room_ac.csv')

In [49]:
print(len(room_ac_epa))
print(len(room_ac_doe))

1323
1943


In [50]:
room_ac_epa.head()

,ENERGY STAR Unique ID,Brand Name,Model Number,Additional Model Information,UPC,Height (inches),Width (inches),Depth (inches),Weight (lbs),Cooling Capacity (Btu/hr),Voltage (volts),Type,Installation Mounting Type,Support Bracket,Heating Mode,Casement Window,Product Class,Variable Speed Compressor,Low Noise,Refrigerant Type,Refrigerant with GWP,Combined Energy Efficiency Ratio (CEER),Percent Less Energy Use than US Federal Standard,Annual Energy Use (kWh/yr),Connected Capable,Connects Using,Communication Hardware Architecture,DR Protocol,Primary Communication Module/Device Brand Name and Model Number,Network Security Standards,Network Standby Power (watts),Is Broadband Internet Connection Needed for Demand Response?,Direct on-premises Open-standard Based Interconnection,Date Available On Market,Date Certified,Markets,CB Model Identifier,Meets ENERGY STAR Most Efficient 2023 Criteria
0,2361168,Black + Decker,BWAM6W**,NaN,NaN,NaN,NaN,NaN,NaN,6000,NaN,Window,NaN,NaN,No,None,"2. Without reverse cycle, with louvered sides,...",No,No,NaN,NaN,12.1,10,372,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/01/2020,06/02/2020,"United States, Canada",ES_1126481_BWAM6W**_06022020111638_80044376,No
1,2361169,Black + Decker,BWAM8W**,NaN,NaN,NaN,NaN,NaN,NaN,8000,NaN,Window,NaN,NaN,No,None,"3. Without reverse cycle, with louvered sides,...",No,No,NaN,NaN,12.0,10,500,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/01/2020,06/02/2020,"United States, Canada",ES_1126481_BWAM8W**_06022020111638_80044376,No
2,2294271,BLACK DECKER,BWAC08WT,NaN,NaN,NaN,NaN,NaN,NaN,8000,NaN,Window,NaN,NaN,No,None,"3. Without reverse cycle, with louvered sides,...",NaN,No,NaN,NaN,12.0,10,500,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04/12/2017,04/12/2017,United States,ES_1126481_BWAC08WT_04132017080959_0999674,No
3,2294272,BLACK DECKER,BWAC10WT,NaN,NaN,NaN,NaN,NaN,NaN,10000,NaN,Window,NaN,NaN,No,None,"3. Without reverse cycle, with louvered sides,...",NaN,No,NaN,NaN,12.0,10,625,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04/12/2017,04/12/2017,United States,ES_1126481_BWAC10WT_04132017081011_1011161,No
4,2294273,BLACK DECKER,BWAC12WT,NaN,NaN,NaN,NaN,NaN,NaN,12000,NaN,Window,NaN,NaN,No,None,"3. Without reverse cycle, with louvered sides,...",NaN,No,NaN,NaN,12.0,10,750,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04/12/2017,04/12/2017,United States,ES_1126481_BWAC12WT_04132017081021_1021858,No


In [52]:
room_ac_doe.head()

,Brand_Name_s__s,Product_Group_Code_Description_s,Basic_Model_Number_m,Individual_Model_Number_Covered_by_Basic_Model_m,Cooling_Capacity__BTU_Hour__d,Electrical_Power_Input__Watts__d,Combined_Energy_Efficiency_Ratio_d,Is_the_Certification_for_this_Basic_Model_Based_on_a_Waiver_of_DOE_s_Test_Procedure_Requirements__s,Is_the_Certification_based_upon_any_Exception_Relief_from_an_Applicable_Standard_by_DOE_s_Office_of_Hearing_and_Appeals__s,Link_to_FTC_EnergyGuide_Label_s
0,Midea,"(11) With reverse cycle, with louvered sides, ...",MWCUPWHP-09HEFN8-BCL1,MWCUPWHP-09HEFN8-BCL1,9000.0,760.0,16.0,No,No,By annual report date
1,Genie Aire,"(8b) Without reverse cycle, without louvered s...",GA23-12ERN1-MCI5,GA23-12ERN1-MCI5,12000.0,1180.0,9.5,No,No,By annual report date
2,Genie Aire,"(8a) Without reverse cycle, without louvered s...",GA23-08CRN1-115,GA23-08CRN1-115,8300.0,760.0,10.6,No,No,By annual report date
3,Genie Aire,"(8b) Without reverse cycle, without louvered s...",GA23-12CRN2-230,GA23-12CRN2-230,12000.0,1120.0,10.5,No,No,By annual report date
4,LG,"(3) Without reverse cycle, with louvered sides...",LW8024RD,LW8024RD,8000.0,710.0,11.4,No,No,http://www.lg.com/global/support/ftclabel


In [63]:
# regexify model number
room_ac_epa['model_number_modified'] = room_ac_epa['Model Number'].apply(regexify_model_number)
room_ac_doe['individual_model_number_modified'] = room_ac_doe['Individual_Model_Number_Covered_by_Basic_Model_m'].apply(regexify_model_number)

In [62]:
# find epa model number in doe database
def find_epa_model_number_in_doe_database(model_number):
    return room_ac_doe['individual_model_number_modified'].str.contains(model_number, regex=True).any()

room_ac_epa_models_in_doe_database = room_ac_epa[room_ac_epa.model_number_modified.apply(find_epa_model_number_in_doe_database)]
room_ac_epa_models_in_doe_database.to_csv('epa_room_ac_models_in_doe_database.csv', index=False)

room_ac_epa_models_not_in_doe_database = room_ac_epa[~room_ac_epa.model_number_modified.apply(find_epa_model_number_in_doe_database)]
room_ac_epa_models_not_in_doe_database.to_csv('epa_room_ac_models_not_in_doe_database.csv', index=False)

print(len(room_ac_epa_models_in_doe_database))
print(len(room_ac_epa_models_not_in_doe_database))

719
604


In [83]:
# Delete part of the string that is in parentheses
def delete_parentheses(model_number):
    return re.sub(r'\(.*\)', '', str(model_number))

room_ac_doe['individual_model_number_modified'] = room_ac_doe['individual_model_number_modified'].apply(delete_parentheses)

In [88]:
room_ac_doe[room_ac_doe['individual_model_number_modified'].str.contains('\(', regex=True)]

# Edit individuall values in the dataframe and replace them.
room_ac_doe.loc[room_ac_doe['individual_model_number_modified'] == 'TWC-06CRD1/L1U(ES', 'individual_model_number_modified'] = 'TWC-06CRD1/L1U'
room_ac_doe.loc[room_ac_doe['individual_model_number_modified'] == 'TWC-08CRD1/L0U(ES)', 'individual_model_number_modified'] = 'TWC-08CRD1/L0U'


In [96]:
# Find doe model number in epa database
def find_doe_models_in_energy_star(model_number):
    return room_ac_epa['model_number_modified'].str.contains(model_number, regex=True).any()

# DOE models that appear in EPA ENERGY STAR dataset
room_ac_doe_energy_star = room_ac_doe[room_ac_doe.individual_model_number_modified.apply(find_doe_models_in_energy_star)]
room_ac_doe_energy_star.to_csv('doe_room_ac_models_in_energy_star_database.csv', index=False)

# DOE models that do not appear in EPA ENERGY STAR dataset
room_ac_doe_non_energy_star = room_ac_doe[~room_ac_doe.individual_model_number_modified.apply(find_doe_models_in_energy_star)]
room_ac_doe_non_energy_star.to_csv('doe_room_ac_models_not_in_energy_star_database.csv', index=False)

print(len(room_ac_doe_energy_star))
print(len(room_ac_doe_non_energy_star))

742
1201


# Clothes Dryer Analysis

In [101]:
clothes_dryer_epa = pd.read_csv('epa_clothes_dryer.csv')

print(len(clothes_dryer_epa))

566


In [102]:
clothes_dryer_doe_1 = pd.read_csv('doe_clothes_dryer_1.csv')
clothes_dryer_doe_2 = pd.read_csv('doe_clothes_dryer_2.csv')

print(len(clothes_dryer_doe_1))
print(len(clothes_dryer_doe_2))

1047
673


In [99]:
# Examine differences between the two DOE datasets
clothes_dryer_doe_1.head()

,Brand_Name_s__s,Product_Group_Code_Description_s,Basic_Model_Number_m,Individual_Model_Number_Covered_by_Basic_Model_m,Capacity__cubic_feet__d,Voltage__volts___For_Electric_Models_Only__d,Hourly_BTU_Rating_of_the_Burner__For_Gas_Models_Only__d,Does_the_Dryer_Have_Automatic_Termination_Controls__s,Combined_Energy_Factor__pounds_per_kilowatt_hour__d,Cycle_Settings_for_Energy_Test_Cycle___Program_Cycle_Setting_s,Cycle_Settings_for_Energy_Test_Cycle___Temperature_Setting_s,Cycle_Settings_for_Energy_Test_Cycle___Dryness_Setting_s,Cycle_Settings_for_Energy_Test_Cycle___Other_Optional_Cycle_Settings_in_the_As_Shipped_Position__If_Applicable__s,Is_the_Certification_for_this_Basic_Model_Based_on_a_Waiver_of_DOE_s_Test_Procedure_Requirements__s,Is_the_Certification_based_upon_any_Exception_Relief_from_an_Applicable_Standard_by_DOE_s_Office_of_Hearing_and_Appeals__s
0,Criterion,"Vented Electric, Standard (4.4 ft3 or greater ...",MDH210-V022/B01EQHS-US,CFDE80N1AW,8.0,240.0,NaN,Yes,3.93,Normal,Medium,Normal Dry +Eco Dry,NaN,No,No
1,Fulgor Milano,"Ventless Electric, Compact (240V) (less than 4...",FM4CD24W1,FM4CD24W1,4.0,240.0,NaN,Yes,2.68,Normal,High,Regular,NaN,No,No
2,Midea,"Vented Electric, Standard (4.4 ft3 or greater ...",MDH210-V022/B01EQHS-US,MH04ED80/W-PR,8.0,240.0,NaN,Yes,3.73,Normal,Medium,Normal dry,NaN,No,No
3,GE Profile,"Vented Electric, Standard (4.4 ft3 or greater ...",M74E_1_R,PTD60EB*R***,7.4,240.0,NaN,Yes,3.93,Normal Cottons,Hot,Dry,ecoDry is default ON for Normal Cottons,No,No
4,GE Profile,Vented Gas,M74G_4_R,PTD60GB*R***,7.4,NaN,22000.0,Yes,3.48,Normal Cottons,Hot,Dry,ecoDry is default ON for Normal Cottons,No,No


In [100]:
clothes_dryer_doe_2.head()

,Brand_Name_s__s,Product_Group_Code_Description_s,Basic_Model_Number_m,Individual_Model_Number_Covered_by_Basic_Model_m,Capacity__cubic_feet__d,Voltage__volts___For_Electric_Models_Only__d,Hourly_BTU_Rating_of_the_Burner__For_Gas_Models_Only__d,Does_the_Dryer_Have_Automatic_Termination_Controls__s,Combined_Energy_Factor__pounds_per_kilowatt_hour__d,Cycle_Settings_for_Energy_Test_Cycle___Program_Cycle_Setting_s,Cycle_Settings_for_Energy_Test_Cycle___Temperature_Setting_s,Cycle_Settings_for_Energy_Test_Cycle___Dryness_Setting_s,Cycle_Settings_for_Energy_Test_Cycle___Other_Optional_Cycle_Settings_in_the_As_Shipped_Position__If_Applicable__s,Is_the_Certification_for_this_Basic_Model_Based_on_a_Waiver_of_DOE_s_Test_Procedure_Requirements__s,Is_the_Certification_based_upon_any_Exception_Relief_from_an_Applicable_Standard_by_DOE_s_Office_of_Hearing_and_Appeals__s
0,Criterion,"Vented Electric, Standard (4.4 ft3 or greater ...",MDH210-V022/B01EQHS-US,CFDE80N1AW,8.0,240.0,NaN,Yes,3.93,Normal,Medium,Normal Dry +Eco Dry,NaN,No,No
1,Fulgor Milano,"Ventless Electric, Compact (240V) (less than 4...",FM4CD24W1,FM4CD24W1,4.0,240.0,NaN,Yes,2.68,Normal,High,Regular,NaN,No,No
2,Midea,"Vented Electric, Standard (4.4 ft3 or greater ...",MDH210-V022/B01EQHS-US,MH04ED80/W-PR,8.0,240.0,NaN,Yes,3.73,Normal,Medium,Normal dry,NaN,No,No
3,GE Profile,"Vented Electric, Standard (4.4 ft3 or greater ...",M74E_1_R,PTD60EB*R***,7.4,240.0,NaN,Yes,3.93,Normal Cottons,Hot,Dry,ecoDry is default ON for Normal Cottons,No,No
4,GE Profile,Vented Gas,M74G_4_R,PTD60GB*R***,7.4,NaN,22000.0,Yes,3.48,Normal Cottons,Hot,Dry,ecoDry is default ON for Normal Cottons,No,No


In [104]:
# Check if doe_1 database contains all the models in doe_2 database
def find_doe_2_models_in_doe_1_database(model_number):
    return clothes_dryer_doe_1['Individual_Model_Number_Covered_by_Basic_Model_m'].str.contains(model_number, regex=True).any()

clothes_dryer_doe_2[clothes_dryer_doe_2.Individual_Model_Number_Covered_by_Basic_Model_m.apply(find_doe_2_models_in_doe_1_database)]


AttributeError: 'DataFrame' object has no attribute 'individual_model_number_modified'